In [25]:
import fiona
import geopandas as gpd
import pandas as pd
import numpy as np
import rtree
from pyspark import SparkContext
from geopandas import GeoDataFrame

In [ ]:
sc = SparkContext()

In [4]:
# import gzip
# import shutil
# with gzip.open('yellow.csv.gz', 'rb') as f_in:
#     with open('yellow.csv', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [37]:
taxi = sc.textFile('yellow.csv.gz')
taxi.take(5)

['tpep_pickup_datetime,tpep_dropoff_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude',
 '2015-02-01 00:00:00.0,2015-02-01 00:10:10.0,40.775485992431641,-73.95855712890625,40.720134735107422,-73.9749755859375',
 '2015-02-01 00:00:00.0,2015-02-01 00:03:36.0,40.728248596191406,-73.984840393066406,40.731391906738281,-73.975341796875',
 '2015-02-01 00:00:00.0,2015-02-01 00:10:32.0,40.731014251708984,-74.001373291015625,40.749237060546875,-74.003067016601562',
 '2015-02-01 00:00:00.0,2015-02-01 00:19:42.0,40.721115112304688,-73.987129211425781,40.794017791748047,-73.96990966796875']

In [43]:
boros = gpd.read_file('boroughs.geojson').to_crs(fiona.crs.from_epsg(2263))
neighborhoods = gpd.read_file('neighborhoods.geojson').to_crs(fiona.crs.from_epsg(2263))

# boros.plot()
# neighborhoods.plot()

In [48]:
boros.head()
boros.iloc[3].boroname
# print(neighborhoods.columns)
# print(boros.columns)

'Manhattan'

In [ ]:

def createIndex(shapefile):
    import rtree
    import fiona.crs
    import geopandas as gpd
    zones = gpd.read_file(shapefile).to_crs(fiona.crs.from_epsg(2263))
    index = rtree.Rtree()
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return {"index": index, "zones": zones}

def findZone(p, geo_map):
    match = geo_map['index'].intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if geo_map['zones'].geometry[idx].contains(p):
            return idx
    return None

def processTrips(pid, records):
    import csv
    import pyproj
    import shapely.geometry as geom

    # Skip the header
    if pid==0:
        next(records)
    
    reader = csv.reader(records)
    counts = {}
    
    # Create an R-tree index
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
    boros = createIndex('boroughs.geojson')    
    neighborhoods = createIndex('neighborhoods.geojson')    
    
    for row in reader:
        # 'tpep_pickup_datetime,tpep_dropoff_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude',
        try: 
            pickup_point = geom.Point(proj(float(row[3]), float(row[2])))
            start_boro = findZone(pickup_point, boros)
            
            if start_boro:
                dropoff_point= geom.Point(proj(float(row[5]), float(row[4])))
                end_hood = findZone(dropoff_point, neighborhoods)
                if end_hood:
                    yield ( (start_boro, end_hood), 1 )
        except: 
            print("Failed at: ", row)
    #                 counts[start_zone] = counts.get(start_zone, 0) + 1
#     return counts.items()


In [ ]:
rdd = sc.textFile('yellow.csv')
counts = rdd.mapPartitionsWithIndex(processTrips) \
            .reduceByKey(lambda x,y: x+y) 
#             .collect()


In [ ]:
print(counts.take(15))